## Generates the poule sheets for Judo+ tournaments  
#### Work by [Gonçalo Sousa](https://github.com/Sousa1909) for [AJDS](https://github.com/AJD-Santarem)  

#### Script 2
---

In [ ]:
import yaml
import os
import pandas as pd
from openpyxl import load_workbook

# Tournament folder
tournament_folder = "REPLACE_ME"

loc_date="Location, date"

# Define output folder
output_folder = "Output/"

# File paths for the Excel templates
best_of_3 = "Templates/BO_3.xlsx"
poule_of_5 = "Templates/Poule_5.xlsx"
double_poule = "Templates/2x_poule.xlsx"

draw_group = "1"
#draw_group = "2"

In [ ]:
# Load the appropriate template based on group size
def load_template(group_size, template_file_2x, template_file_5, template_file_2):
    if group_size == 2:
        return "bo3", load_workbook(template_file_2)
    if group_size > 5:
        return "2x", load_workbook(template_file_2x)
    else:
        return "standard", load_workbook(template_file_5)

# Populate the table with the group information
def populate_group_table(group_data, template_file_2x, template_file_5, template_file_2, output_file, group_number, age_tier, year):
    # Determine group size based on the number of athletes in the group
    sheet_type, workbook = load_template(len(group_data), template_file_2x, template_file_5, template_file_2)
    sheet = workbook.active

    # Fill in athlete data based on the loaded template
    if sheet_type == "bo3":
        sheet['B3'] = loc_date
        for i, athlete in enumerate(group_data, start=8):
            sheet[f'C{i}'] = athlete['Name']
            sheet[f'D{i}'] = athlete['Club']
        sheet[f'F{13}'] = group_number
        sheet[f'J{4}'] = age_tier + " " + year
    elif sheet_type == "standard":
        sheet['C3'] = loc_date
        for i, athlete in enumerate(group_data, start=9):
            sheet[f'B{i}'] = athlete['Name']
            sheet[f'C{i}'] = athlete['Club']
        sheet[f'L{17}'] = group_number
        sheet[f'M{4}'] = age_tier + " " + year
    elif sheet_type == "2x":
        sheet['C3'] = loc_date
        start_A = 8
        start_b = 24
        for i, athlete in enumerate(group_data):
            if i % 2 == 0:
                # Write in B8, B9, B10, B11, B12
                row = start_A + (i // 2)
            else:
                # Write in B24, B25, B26, B27, B28
                row = start_b + (i // 2)
            sheet[f'B{row}'] = athlete['Name']
            sheet[f'C{row}'] = athlete['Club']
        sheet[f'M{2}'] = group_number
        sheet[f'M{4}'] = age_tier + " " + year
    else:
        print(f"--ERROR-- Group {group_number} for {age_tier} {year} failed to be passed to xlsx")
        pass

    # Save the updated file to the designated folder
    output_filename = os.path.join(output_folder + tournament_folder, output_file.format(age_tier, group_number))
    workbook.save(output_filename)
    print(f"Group {group_number} saved to {output_filename}")

# Function to read groups from the YAML file
def read_groups_from_yaml(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)

    # Extract groups by age tier
    groups_by_age_tier = []
    for age_tier, groups in data.items():
        age_tier_groups = []
        for group_info in groups:
            # Extract group key (e.g., 'Group 1') and its athletes
            for group_key, athletes in group_info.items():
                age_tier_groups.append(pd.DataFrame(athletes))
        groups_by_age_tier.append((age_tier, age_tier_groups))

    return groups_by_age_tier

# Function to process each group and generate tables from YAML data
def create_tables_from_yaml(yaml_file_path, template_file_2x, template_file_5, template_file_2, output_file_pattern, year=""):
    groups_by_age_tier = read_groups_from_yaml(yaml_file_path)
    for age_tier, groups in groups_by_age_tier:
        group_number = 1
        for group in groups:
            # Populate and save each group table
            populate_group_table(group.to_dict(orient='records'), template_file_2x, template_file_5, template_file_2, output_file_pattern, group_number, age_tier, year)
            group_number += 1

# Generate tables for both masculine and feminine groups
if draw_group == "1":
    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_15_16_masc.yaml', double_poule, poule_of_5, best_of_3, 'Group_15_16_M_{}_{}.xlsx', "2015 / 2016")
    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_15_16_fem.yaml', double_poule, poule_of_5, best_of_3, 'Group_15_16_F_{}_{}.xlsx', "2015 / 2016")

    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_17_18_masc.yaml', double_poule, poule_of_5, best_of_3, 'Group_17_18_M_{}_{}.xlsx', "2017 / 2018")
    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_17_18_fem.yaml', double_poule, poule_of_5, best_of_3, 'Group_17_18_F_{}_{}.xlsx', "2017 / 2018")
else:
    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_IN_masc.yaml', double_poule, poule_of_5, best_of_3, 'Group_IN_M_{}_{}.xlsx')
    create_tables_from_yaml(f'{output_folder + tournament_folder}/.grouped_athletes_IN_fem.yaml', double_poule, poule_of_5, best_of_3, 'Group_IN_F_{}_{}.xlsx')